In [5]:
import sqlite3
import os
import pandas as pd

# Connect to the database
conn = sqlite3.connect('dogs.db')
cursor = conn.cursor()

# Create tables
cursor.execute('''CREATE TABLE IF NOT EXISTS Breeds
                (breed_id INTEGER PRIMARY KEY, breed_name TEXT)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Images
                (image_id INTEGER PRIMARY KEY, file_path TEXT, breed_id INTEGER,
                FOREIGN KEY(breed_id) REFERENCES Breeds(breed_id))''')

# Function to get breed ID from breed name or insert new breed and get its ID
def get_or_insert_breed_id(breed_name):
    cursor.execute("SELECT breed_id FROM Breeds WHERE breed_name = ?", (breed_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        cursor.execute("INSERT INTO Breeds (breed_name) VALUES (?)", (breed_name,))
        conn.commit()
        return cursor.lastrowid

# Function to traverse directory structure and insert image data into database
def insert_image_data(directory):
    for breed_name in os.listdir(directory):
        breed_directory = os.path.join(directory, breed_name)
        if os.path.isdir(breed_directory):
            breed_id = get_or_insert_breed_id(breed_name)
            for filename in os.listdir(breed_directory):
                if filename.endswith(".jpg"):
                    file_path = os.path.join(breed_directory, filename)
                    cursor.execute("INSERT INTO Images (file_path, breed_id) VALUES (?, ?)", (file_path, breed_id))
    conn.commit()

# Insert image data into database
insert_image_data('/Users/kalebdecker/Documents/GitHub/Project-4-Group-3/Data/Resources')

# Query data from Images table to verify
cursor.execute("SELECT * FROM Images")
image_data = cursor.fetchall()

# Convert to DataFrame for further inspection
df = pd.DataFrame(image_data, columns=['image_id', 'file_path', 'breed_id'])
print("Image DataFrame:")
print(df)

# Close connection
conn.close()





Image DataFrame:
Empty DataFrame
Columns: [image_id, file_path, breed_id]
Index: []
